In [19]:
import pandas as pd   
from matplotlib import pyplot as plt
from numpy import mean
import time

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import PolynomialFeatures

import warnings 
warnings.filterwarnings('ignore')


from sklearn.linear_model import LinearRegression

In [20]:
df1 = pd.read_csv("train.csv")
#df1.head()
df2 = pd.read_csv('test.csv')
#df2

In [ ]:
#from df1 drop sub_are
df1 = df1.drop(columns=['sub_area'])


In [ ]:
#from df2 drop sub_area and row ID 
df2 = df2.drop(columns=['sub_area', 'row ID'])

In [21]:
df2 = df2.drop(columns=[ 'row ID'])

In [22]:
category_frequencies = df1['sub_area'].value_counts()

print(category_frequencies)

Poselenie Shhapovskoe             1105
Alekseevskoe                      1104
Tverskoe                          1101
Poselenie Mosrentgen              1099
Caricyno                          1097
                                  ... 
Posleenie Novofedorovskoe            3
Poseleine Voronovskoe                3
Poselenie Mhiajlovo-Jarcevskoe       3
Prsopekt Vernadskogo                 2
Poselenie Krasnopahorksoe            2
Name: sub_area, Length: 1927, dtype: int64


In [23]:
infrequent_categories = category_frequencies[category_frequencies < 900].index.tolist()
print(len(infrequent_categories))
print(len(category_frequencies))

1781
1927


In [24]:
df1['sub_area'] = df1['sub_area'].replace(infrequent_categories, 'Other')
df2['sub_area'] = df2['sub_area'].replace(infrequent_categories, 'Other')
category_frequencies1 = df1['sub_area'].value_counts()
print(len(category_frequencies1))

147


In [ ]:
df_onehot = pd.get_dummies(df1)
df_onehot.dtypes
test_onehot = pd.get_dummies(df2)

In [25]:
label_encoder = LabelEncoder()
categorical_cols = df1.select_dtypes(include=['object']).columns.tolist()
categorical_colstest = df2.select_dtypes(include=['object']).columns.tolist()

# You can also encode labels to identify categorical columns
for col in categorical_cols:
    df1[col] = label_encoder.fit_transform(df1[col])

for col in categorical_colstest:
    df2[col] = label_encoder.fit_transform(df2[col])


In [ ]:
print

In [ ]:
X = df_onehot.drop(columns=['price_doc'])
print(X.shape)
y = df_onehot[['price_doc']]






In [26]:
X = df1.drop(columns=['price_doc'])
print(X.shape)
y = df1[['price_doc']]






(181507, 271)


In [28]:
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)
X_testscaled = scaler.fit_transform(df2)

In [ ]:
#drop row id from x_testscaled
X_testscaled = X_testscaled[:, 1:]


In [ ]:
print(X_testscaled.shape)

In [29]:
#covert x_scaled to dataframe
X_scaled = pd.DataFrame(X_scaled)
#convert x_testscaled to dataframe
X_testscaled = pd.DataFrame(X_testscaled)


In [ ]:
print(X_scaled.shape)

In [ ]:
X_testscaled = X_testscaled[X_scaled.columns]

In [31]:
from sklearn.decomposition import PCA
n_components = 150  # You can choose the number of components based on your requirements
pca = PCA(n_components=n_components)

# Apply PCA to the training data
X_pca = pca.fit_transform(X_scaled)

# Apply the same PCA transformation to the test data
df2_onehot_pca = pca.transform(X_testscaled)

In [39]:
print(df2_onehot_pca.shape)

(77789, 150)


In [32]:
ols_reg = LinearRegression()
sfs = SequentialFeatureSelector(ols_reg, direction='forward', n_features_to_select=25,n_jobs=-1)
sfs.fit(X_pca, y)
#X_selected = sfs.transform(X_pca)
# Print the names of the selected features
print("Selected Features:", sfs.get_feature_names_out())




Selected Features: ['x0' 'x1' 'x3' 'x4' 'x7' 'x9' 'x10' 'x15' 'x16' 'x17' 'x21' 'x22' 'x23'
 'x31' 'x36' 'x41' 'x48' 'x70' 'x74' 'x86' 'x87' 'x99' 'x101' 'x128'
 'x134']


In [37]:
#save the selected features in a dataframe
X_selected = pd.DataFrame(sfs.transform(X_pca), columns=sfs.get_feature_names_out())

In [42]:
#save the selected features in a dataframe
X_testselected = pd.DataFrame(sfs.transform(df2_onehot_pca), columns=sfs.get_feature_names_out())

In [41]:
X_selected.shape

(181507, 25)

In [43]:
X_testselected.shape

(77789, 25)

In [40]:
X_testscaled = X_testscaled[X_selected.columns]

KeyError: "['x101', 'x128', 'x134'] not in index"

In [ ]:
print(X_testscaled.info())


In [38]:
# Create a mapping dictionary for the old column names to the new names
column_mapping = {i: f'x{i}' for i in range(100)}

# Rename the columns in X_testscaled
X_testscaled.rename(columns=column_mapping, inplace=True)

# Get the names of the selected features
selected_feature_names = sfs.get_feature_names_out()

# Filter columns in X_testscaled based on selected features
X_testselected = X_testscaled[selected_feature_names]


KeyError: "['x101', 'x128', 'x134'] not in index"

In [ ]:
#covert x_scaled to dataframe
X_scaled = pd.DataFrame(X_pca)
#convert x_testscaled to dataframe
X_testscaled = pd.DataFrame(df2_onehot_pca)

In [ ]:
print(X_testselected.shape)

In [ ]:
#give me code for feature importance using linear regression
model = LinearRegression()
model.fit(X_scaled, y)
# get importance
importance = model.coef_[0]
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

# drop features with low importance
threshold = 70000  # Adjust the threshold as needed
selected_features = [feature for feature, score in zip(X_scaled.columns, importance) if score >= threshold]
X_selected = X_scaled[selected_features]

# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()



In [ ]:
import statsmodels.api as sm
import pandas as pd

# Convert scaled_df1 to a DataFrame without specifying columns

# Add a constant term to the feature matrix
X_with_const = sm.add_constant(X_scaled)

# Fit a linear regression model
model = sm.OLS(y, X_with_const).fit()

# Get p-values for each feature
p_values = model.pvalues[1:]  # Exclude the constant term

# Set your desired threshold for p-value
threshold = 0.05

# Filter features based on p-value
selected_features = p_values[p_values < threshold].index

# Display selected features
print("Selected Features:")
print(selected_features)

# Select columns in the DataFrame
X_selected = X_scaled[selected_features]


In [ ]:
# original_feature_indices = np.arange(X.shape[1])
threshold_value = 0.001
variance_filter = VarianceThreshold(threshold=threshold_value)

# Fit and transform the feature matrix
X_filtered = variance_filter.fit_transform(X_pca)

X_test_high_variance = variance_filter.transform(df2_onehot_pca)

In [ ]:
print(X_selected.shape)

In [ ]:
print(X_testscaled.shape)

In [ ]:
print(X_selected.head())

In [ ]:
print(X_filtered.shape)

In [ ]:
print(X_test_high_variance.shape)

In [ ]:
#store x_filtered as a df
X_filtered = pd.DataFrame(X_filtered)

In [ ]:
X_selected = pd.DataFrame(X_selected)

In [ ]:
#keep those columns in X_testscaled that are in X_selected
X_testscaled = X_testscaled[X_selected.columns]

In [ ]:
# Make sure column names are trimmed

# Find the common columns
common_columns = X_testscaled.columns.intersection(X_selected.columns)

# Keep only the common columns in X_testscaled
X_testscaled_filtered = X_testscaled[common_columns]

# Now X_testscaled_filtered contains only the columns that are in both X_testscaled and X_selected


In [ ]:
common_columns = X_testscaled.columns.intersection(X_selected.columns)
X_testscaled = X_testscaled[common_columns]


In [ ]:
print(X_testscaled.shape)


In [ ]:
# Get the common column names between X_testselected and X_selected
common_columns = X_testscaled.columns.intersection(X_selected.columns)

# Keep only the common columns in X_testselected
X_testselected = X_testscaled[X_selected.columns.intersection(common_columns)]


In [ ]:
print(X_testselected.shape)

In [ ]:
print(X_testscaled_filtered.head())

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_selected)
test_scaled = scaler.transform(test_onehot)
print(test_scaled.shape)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
print("Training Set Info:")
print(X.info())

print("\nTest Set Info:")
print(test_onehot.info())


In [44]:
poly = PolynomialFeatures(3, interaction_only=True)
X2 = poly.fit_transform(X_selected)
test2 = poly.fit_transform(X_testselected)
print(X2.shape)
print(poly.get_feature_names_out())
reg2 = LinearRegression().fit(X2, y)
print(reg2.score(X2, y))
print(reg2.coef_)
print(reg2.intercept_)
md_probs = reg2.predict(test2)

(181507, 2626)
['1' 'x0' 'x1' ... 'x99 x101 x134' 'x99 x128 x134' 'x101 x128 x134']
0.6657085569351444
[[-1.73162865e-08  2.98909053e+05 -2.02469769e+05 ...  3.40440340e+04
   7.85579495e+03 -1.45958274e+04]]
[16438256.0858673]


In [45]:
additional_data = pd.read_csv("test.csv")

# Create a DataFrame with 'row ID' and predicted values
predicted_df = pd.DataFrame({
    'row ID': additional_data['row ID'],
    'price_doc': md_probs[:,0]  # Replace with the actual column name for your predictions
})

# Save the DataFrame to a new CSV file
predicted_df.to_csv('predictions36.csv', index=False)